In [1]:
from sqlalchemy import create_engine
import json as js
import pandas as pd

In [2]:
credenciales = js.load(open('C:/Users/javis/OneDrive/Escritorio/HelloDevOps/credentials.json','rb'))

usuario = credenciales['user']
pwd = credenciales['pwd']
host = credenciales['host']
port = credenciales['port']
db = credenciales['db']

engine = create_engine(f'mysql+pymysql://{usuario}:{pwd}@{host}:{port}/{db}').connect()

In [3]:
micro = pd.read_csv('C:/Users/javis/Downloads/godaddy-microbusiness-density-forecasting/train.csv')
micro

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [4]:
cat_state = micro['state'].drop_duplicates().reset_index(drop=True).reset_index().rename({'index':'id_state'},axis=1)

In [5]:
cat_county = micro[['cfips','county']].drop_duplicates()
cat_county

,cfips,county
0,1001,Autauga County
39,1003,Baldwin County
78,1005,Barbour County
117,1007,Bibb County
156,1009,Blount County
...,...,...
122070,56037,Sweetwater County
122109,56039,Teton County
122148,56041,Uinta County
122187,56043,Washakie County


In [6]:
micro1 = micro.copy()
micro1 = micro1.merge(cat_state,on='state')
micro1

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,id_state
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,0
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,0
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,0
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,0
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,0
...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,50
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,50
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,50
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,50


In [7]:
hechos = micro1[['cfips','id_state','first_day_of_month','microbusiness_density','active']]
hechos

,cfips,id_state,first_day_of_month,microbusiness_density,active
0,1001,0,2019-08-01,3.007682,1249
1,1001,0,2019-09-01,2.884870,1198
2,1001,0,2019-10-01,3.055843,1269
3,1001,0,2019-11-01,2.993233,1243
4,1001,0,2019-12-01,2.993233,1243
...,...,...,...,...,...
122260,56045,50,2022-06-01,1.803249,101
122261,56045,50,2022-07-01,1.803249,101
122262,56045,50,2022-08-01,1.785395,100
122263,56045,50,2022-09-01,1.785395,100


In [8]:
micro1.to_sql(con=engine, name='hechos', if_exists='replace', index=False)
cat_state.to_sql(con=engine, name='estados', if_exists='replace', index=False)
cat_county.to_sql(con=engine, name='county', if_exists='replace', index=False)


3135

In [9]:
engine.close()